In [1]:
from google.colab import files
uploaded = files.upload()


Saving restaurants.sql to restaurants (1).sql
Saving users.json to users (1).json
Saving orders.csv to orders (1).csv


In [2]:

import pandas as pd
import json
import sqlite3

In [3]:
orders = pd.read_csv('orders.csv')
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:

with open('users.json', 'r') as f:
    users_data = json.load(f)


In [5]:
if isinstance(users_data, list):
    users = pd.DataFrame(users_data)
elif isinstance(users_data, dict):
    if 'users' in users_data:
        users = pd.DataFrame(users_data['users'])
    else:
        users = pd.DataFrame.from_dict(users_data, orient='index')
        users.reset_index(inplace=True)
        users.rename(columns={'index': 'user_id'}, inplace=True)

In [6]:

users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [7]:

conn = sqlite3.connect(':memory:')

with open('restaurants.sql', 'r') as f:
    sql_script = f.read()
    conn.executescript(sql_script)


In [8]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
table_name = tables[0][0] if tables else 'restaurants'

In [9]:

restaurants = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
conn.close()

In [10]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [11]:
merged_step1 = pd.merge(orders, users, on='user_id', how='left', suffixes=('', '_user'))
final_dataset = pd.merge(merged_step1, restaurants, on='restaurant_id', how='left', suffixes=('', '_restaurant'))


In [12]:

final_dataset.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,restaurant_name_restaurant,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [13]:

print(f"Total Records: {len(final_dataset)}")
print(f"Total Columns: {len(final_dataset.columns)}")
print(f"Unique Users: {final_dataset['user_id'].nunique()}")
print(f"Unique Restaurants: {final_dataset['restaurant_id'].nunique()}")


Total Records: 10000
Total Columns: 12
Unique Users: 2883
Unique Restaurants: 500


In [14]:
final_dataset.isnull().sum()

final_dataset.to_csv('final_food_delivery_dataset.csv', index=False)

from google.colab import files
files.download('final_food_delivery_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>